In [ ]:
# 导入所需库
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime

In [ ]:
# 导入数据文件（excel）
# 6month.xlsx是6月份用户订单数据，7month.xlsx是7月份用户订单数据，注册用户数据.xls是用户信息数据
df_6 = pd.read_excel("6month.xlsx")
df_7 = pd.read_excel("7month.xlsx")
df_users = pd.read_excel(u"注册用户数据.xls")

In [ ]:
# 定义时间类型变量
six_m = pd.to_datetime('2017-06-01')
senven_m = pd.to_datetime('2017-07-01')
eight_m = pd.to_datetime('2017-08-01')

In [ ]:
# 在用户数据表中筛选6月（7月）注册用户
user_6 = df_users[(df_users[u'注册时间'] >= six_m) & (df_users[u'注册时间'] < senven_m)][[u'注册时间',u'药店名称']]
user_7 = df_users[(df_users[u'注册时间'] >= senven_m) & (df_users[u'注册时间'] < eight_m)][[u'注册时间',u'药店名称']]

In [ ]:
# 连接筛选后的表和6月（7月）订单表，找出6月（7月）注册用户的订单
sheet_6 = pd.merge(df_6, user_6, left_on=u'下单用户', right_on=u'药店名称')
sheet_7 = pd.merge(df_7, user_7, left_on=u'下单用户', right_on=u'药店名称')

In [ ]:
# 修改时间数据类型，便于进行时间比较
sheet_6[u'下单时间'] = pd.to_datetime(sheet_6[u'下单时间'])
sheet_7[u'下单时间'] = pd.to_datetime(sheet_7[u'下单时间'])

In [ ]:
def top(df, n=3, column=u'下单时间'):
    return df.sort_values(by=column)[0:n]

In [ ]:
s6 = sheet_6.groupby(u'下单用户').apply(top)

In [ ]:
# 导出表格到excel文件
writer = pd.ExcelWriter('output.xlsx')
sheet_6.to_excel(writer,u'6月新用户订单')
sheet_7.to_excel(writer,u'7月新用户订单')
s6.to_excel(writer,u'6月新用户前三笔订单')
s7.to_excel(writer,u'7月新用户前三笔订单')
writer.save()

In [ ]:
# 分别获取每个用户前三笔数据

In [ ]:
def get_one(df, n=0, column=u'下单时间'):
    return df.sort_values(by=column)[n:n+1]

In [ ]:
s6_1 = sheet_6.groupby(u'下单用户').apply(get_one, n=0)
s6_2 = sheet_6.groupby(u'下单用户').apply(get_one, n=1)
s6_3 = sheet_6.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
s7_1 = sheet_7.groupby(u'下单用户').apply(get_one, n=0)
s7_2 = sheet_7.groupby(u'下单用户').apply(get_one, n=1)
s7_3 = sheet_7.groupby(u'下单用户').apply(get_one, n=2)

In [ ]:
writer = pd.ExcelWriter('output.xlsx')
s6_1.to_excel(writer,u'6月新用户第一笔订单')
s6_2.to_excel(writer,u'6月新用户第二笔订单')
s6_3.to_excel(writer,u'6月新用户第三笔订单')
s7_1.to_excel(writer,u'7月新用户第一笔订单')
s7_2.to_excel(writer,u'7月新用户第二笔订单')
s7_3.to_excel(writer,u'7月新用户第三笔订单')
writer.save()